In [21]:
# 1. 데이터
import numpy as np
x1 = np.array([range(100), range(311,411), range(100)])
y1 = np.array([range(501,601), range(711,811), range(100)])
 
x2 = np.array([range(100,200), range(311,411), range(100,200)])
y2 = np.array([range(501,601), range(711,811), range(100)])
 
x1 = np.transpose(x1)
y1 = np.transpose(y1)
x2 = np.transpose(x2)
y2 = np.transpose(y2)
 
print('x1s shape : ', x1.shape)
print('x2s shape : ', x2.shape)
print('y1s shape : ', y1.shape)
print('y2s shape : ', y2.shape)
 
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=66, test_size=0.4)
x1_test, x1_val, y1_test, y1_val = train_test_split(x1_test, y1_test, random_state=66, test_size=0.5)
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, random_state=66, test_size=0.4)
x2_test, x2_val, y2_test, y2_val = train_test_split(x2_test, y2_test, random_state=66, test_size=0.5)
 
# 2. 2개의 모델 구성
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
 
## model 1
input1 = Input(shape=(3,)) 
dense1 = Dense(5, activation='relu')(input1) 
dense2 = Dense(3)(dense1) 
dense3 = Dense(4)(dense2)
middle1 = Dense(3)(dense3)
 
## model 2
input2 = Input(shape=(3,)) 
dense1 = Dense(5, activation='relu')(input2) 
dense2 = Dense(3)(dense1) 
dense3 = Dense(4)(dense2)
middle2 = Dense(3)(dense3)
 
# 모델 합치기 concatenate
from tensorflow.keras.layers import concatenate
#tf.keras.layers.concatenate
merge1 = concatenate([middle1, middle2]) 
 
output1 = Dense(30)(merge1)
output1 = Dense(13)(output1)
output1 = Dense(3)(output1)
 
output2 = Dense(15)(merge1)
output2 = Dense(32)(output2)
output2 = Dense(3)(output2)
 
model = Model(inputs=[input1, input2], outputs=[output1, output2])
 
# 3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit([x1_train, x2_train],[y1_train, y2_train], epochs=100, batch_size=1, validation_data=([x1_val, x2_val],[y1_val, y2_val])) 
# input, output para 2
 
# 4. 평가 예측
mse = model.evaluate([x1_test, x2_test], [y1_test, y2_test], batch_size=1)
# in, out, merge 모델이 5개....라서 mse 5개?
print('loss(mse) : ', mse) # compile에서 lose = mse, loss와 mse둘다 만들 필요 없음
print('loss(mse) : ', mse[0])
print('loss(mse) : ', mse[1])
print('loss(mse) : ', mse[2])
print('loss(mse) : ', mse[3])
print('loss(mse) : ', mse[4])
 
y1_predict, y2_predict = model.predict([x1_test, x2_test])
print('PREDICT : ', y1_predict, y2_predict) #RMSE와 R2를 위해 쪼개기
 
# RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
 
RMSE1=RMSE(y1_test, y1_predict)
RMSE2=RMSE(y2_test, y2_predict)
print('RMSE(y1_test) : ', RMSE1)
print('RMSE(y2_test) : ', RMSE2)
print('AVG(RMSE) : ', (RMSE1+RMSE2)/2)
 
# R2 구하기
from sklearn.metrics import r2_score
def R2(y_test, y_predict):
    return r2_score(y_test, y_predict)
 
R2_1 = R2(y1_test, y1_predict)
R2_2 = R2(y2_test, y2_predict)
print('R2(y1_test) : ', R2_1)
print('R2(y2_test) : ', R2_2)
print('AVG(R2) : ', (R2_1+R2_2)/2)


x1s shape :  (100, 3)
x2s shape :  (100, 3)
y1s shape :  (100, 3)
y2s shape :  (100, 3)
Train on 60 samples, validate on 20 samples
Epoch 1/100
60/60 [==============================] - 2s 27ms/sample - loss: 377554.5553 - dense_78_loss: 174663.5625 - dense_81_loss: 202891.0312 - dense_78_mse: 174663.5625 - dense_81_mse: 202891.0312 - val_loss: 44192.5743 - val_dense_78_loss: 15566.1816 - val_dense_81_loss: 28626.3906 - val_dense_78_mse: 15566.1816 - val_dense_81_mse: 28626.3906
Epoch 2/100
60/60 [==============================] - 0s 8ms/sample - loss: 18225.6924 - dense_78_loss: 8971.0947 - dense_81_loss: 9254.5977 - dense_78_mse: 8971.0947 - dense_81_mse: 9254.5977 - val_loss: 12459.7015 - val_dense_78_loss: 6274.2480 - val_dense_81_loss: 6185.4536 - val_dense_78_mse: 6274.2480 - val_dense_81_mse: 6185.4536
Epoch 3/100
60/60 [==============================] - 0s 8ms/sample - loss: 13015.0719 - dense_78_loss: 6508.9204 - dense_81_loss: 6506.1528 - dense_78_mse: 6508.9204 - dense_81_mse

60/60 [==============================] - 0s 8ms/sample - loss: 0.0022 - dense_78_loss: 0.0011 - dense_81_loss: 0.0011 - dense_78_mse: 0.0011 - dense_81_mse: 0.0011 - val_loss: 0.0613 - val_dense_78_loss: 0.0305 - val_dense_81_loss: 0.0308 - val_dense_78_mse: 0.0305 - val_dense_81_mse: 0.0308
Epoch 27/100
60/60 [==============================] - 0s 8ms/sample - loss: 0.0021 - dense_78_loss: 0.0011 - dense_81_loss: 0.0010 - dense_78_mse: 0.0011 - dense_81_mse: 0.0010 - val_loss: 0.0490 - val_dense_78_loss: 0.0244 - val_dense_81_loss: 0.0246 - val_dense_78_mse: 0.0244 - val_dense_81_mse: 0.0246
Epoch 28/100
60/60 [==============================] - 0s 8ms/sample - loss: 9.1598e-04 - dense_78_loss: 4.5919e-04 - dense_81_loss: 4.5679e-04 - dense_78_mse: 4.5919e-04 - dense_81_mse: 4.5679e-04 - val_loss: 0.0439 - val_dense_78_loss: 0.0219 - val_dense_81_loss: 0.0221 - val_dense_78_mse: 0.0219 - val_dense_81_mse: 0.0221
Epoch 29/100
60/60 [==============================] - 0s 8ms/sample - loss:

60/60 [==============================] - 0s 8ms/sample - loss: 9.4673e-09 - dense_78_loss: 3.6600e-09 - dense_81_loss: 5.8073e-09 - dense_78_mse: 3.6600e-09 - dense_81_mse: 5.8073e-09 - val_loss: 0.0310 - val_dense_78_loss: 0.0155 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0155 - val_dense_81_mse: 0.0155
Epoch 52/100
60/60 [==============================] - 0s 8ms/sample - loss: 9.5407e-09 - dense_78_loss: 3.7070e-09 - dense_81_loss: 5.8337e-09 - dense_78_mse: 3.7070e-09 - dense_81_mse: 5.8337e-09 - val_loss: 0.0310 - val_dense_78_loss: 0.0155 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0155 - val_dense_81_mse: 0.0155
Epoch 53/100
60/60 [==============================] - 0s 8ms/sample - loss: 8.8788e-09 - dense_78_loss: 3.9390e-09 - dense_81_loss: 4.9398e-09 - dense_78_mse: 3.9390e-09 - dense_81_mse: 4.9398e-09 - val_loss: 0.0310 - val_dense_78_loss: 0.0155 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0155 - val_dense_81_mse: 0.0155
Epoch 54/100
60/60 [===============

60/60 [==============================] - 0s 8ms/sample - loss: 1.9114e-08 - dense_78_loss: 9.7001e-09 - dense_81_loss: 9.4139e-09 - dense_78_mse: 9.7001e-09 - dense_81_mse: 9.4139e-09 - val_loss: 0.0310 - val_dense_78_loss: 0.0154 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0154 - val_dense_81_mse: 0.0155
Epoch 77/100
60/60 [==============================] - 0s 8ms/sample - loss: 2.6641e-08 - dense_78_loss: 1.5114e-08 - dense_81_loss: 1.1527e-08 - dense_78_mse: 1.5114e-08 - dense_81_mse: 1.1527e-08 - val_loss: 0.0310 - val_dense_78_loss: 0.0155 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0155 - val_dense_81_mse: 0.0155
Epoch 78/100
60/60 [==============================] - 0s 8ms/sample - loss: 1.1906e-08 - dense_78_loss: 5.2380e-09 - dense_81_loss: 6.6677e-09 - dense_78_mse: 5.2380e-09 - dense_81_mse: 6.6677e-09 - val_loss: 0.0310 - val_dense_78_loss: 0.0154 - val_dense_81_loss: 0.0155 - val_dense_78_mse: 0.0154 - val_dense_81_mse: 0.0155
Epoch 79/100
60/60 [===============

loss(mse) :  [8.546689571176103e-07, 4.5752168e-07, 3.9714723e-07, 4.5752168e-07, 3.9714723e-07]
loss(mse) :  8.546689571176103e-07
loss(mse) :  4.5752168e-07
loss(mse) :  3.9714723e-07
loss(mse) :  4.5752168e-07
loss(mse) :  3.9714723e-07
PREDICT :  [[5.0499902e+02 7.1499872e+02 4.0000134e+00]
 [5.9499963e+02 8.0499969e+02 9.3999634e+01]
 [5.3899927e+02 7.4899908e+02 3.7999836e+01]
 [5.0899899e+02 7.1899878e+02 7.9999437e+00]
 [5.7299951e+02 7.8299939e+02 7.1999710e+01]
 [5.8499957e+02 7.9499969e+02 8.3999512e+01]
 [5.0099893e+02 7.1099866e+02 9.3467534e-06]
 [5.1899908e+02 7.2899890e+02 1.7999958e+01]
 [5.3399921e+02 7.4399902e+02 3.2999863e+01]
 [5.2499921e+02 7.3499902e+02 2.3999931e+01]
 [5.4399933e+02 7.5399927e+02 4.2999817e+01]
 [5.7399951e+02 7.8399945e+02 7.2999626e+01]
 [5.8099957e+02 7.9099957e+02 7.9999626e+01]
 [5.9299976e+02 8.0299969e+02 9.1999580e+01]
 [5.2599908e+02 7.3599896e+02 2.4999832e+01]
 [5.8599945e+02 7.9599963e+02 8.4999550e+01]
 [5.1499908e+02 7.2499884e+02